Step 1: load the house-prices-advanced-regression-techniques dataset.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
  
# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/train.csv


step 2:  
set **X_full** as full train set  
set **X_test_full** as full test set  
set y to target -- SalePrice  
X_train_full, X_valid_full, y_train, y_valid




In [2]:
X_full = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
X_test_full = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')


X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2,random_state=0)



finish with setting

In [3]:
X_train_full.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
618,619,20,RL,90.0,11694,Pave,NaN,Reg,Lvl,AllPub,...,260,0,NaN,NaN,NaN,0,7,2007,New,Partial
870,871,20,RL,60.0,6600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2009,WD,Normal
92,93,30,RL,80.0,13360,Pave,Grvl,IR1,HLS,AllPub,...,0,0,NaN,NaN,NaN,0,8,2009,WD,Normal
817,818,20,RL,NaN,13265,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal
302,303,20,RL,118.0,13704,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,1,2006,WD,Normal


I want to check missing values for only numerical predictors

In [4]:
X_num = X_full.select_dtypes(exclude=['object'])
X_test_num = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train_num, X_valid_num, y_train_num, y_valid_num = train_test_split(X_num, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

check the missing values

In [5]:
# Shape of training data (num_rows, num_columns)
print(X_train_num.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train_num.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 37)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


Missing values columns miss less than 20%, I don't need to drop them.  
check "Cardinality" for categorical columns, I wanna know how many cardinality for each categorial columns. 

In [6]:
# Get number of unique entries in each column with categorical data
object_cols = [col for col in X_train_full.columns if X_train_full[col].dtype == "object"]
object_nunique = list(map(lambda col: X_train_full[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Alley', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('GarageFinish', 3),
 ('PavedDrive', 3),
 ('PoolQC', 3),
 ('MiscFeature', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('MasVnrType', 4),
 ('ExterQual', 4),
 ('BsmtQual', 4),
 ('BsmtCond', 4),
 ('BsmtExposure', 4),
 ('KitchenQual', 4),
 ('Fence', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Electrical', 5),
 ('FireplaceQu', 5),
 ('GarageQual', 5),
 ('GarageCond', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('BsmtFinType1', 6),
 ('BsmtFinType2', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('GarageType', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [7]:
cardin = pd.DataFrame(sorted(d.items(), key=lambda x: x[1]))
print(cardin.shape)


(43, 2)


In [8]:
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

In [9]:
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train_new = X_train_full[my_cols].copy()
X_valid_new = X_valid_full[my_cols].copy()
X_test_new = X_test_full[my_cols].copy()

In [10]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators = 100, random_state = 0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train_new, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid_new)

print('MAE:', mean_absolute_error(y_valid, preds))


MAE: 17741.26297945205


when model n_estimators = 100, random_state = 0), MAE:17687, I would like to know if there is better result when I use different n_estmators.

In [11]:
model = RandomForestRegressor(n_estimators = 350, random_state = 0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train_new, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid_new)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17594.641996086106


In [12]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test_new)

In [13]:
samplesubmission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

output = pd.DataFrame({'Id': samplesubmission.Id, 'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)